# Mini-projet: calcul d'itinéraire routier

On donne la carte routière simplifiée suivante:

![carte routière simplifiée](https://snlpdo.fr/tnsi/img/07-carte_routiere.png)

## I. Version console

L'objectif est d'écrire un code Python qui demande à l'utilisateur de saisir les villes de départ et d'arrivée puis qui calcule et affiche le meilleur itinéraire (avec toutes ses étapes).

### 1. Représentation des données

Quelle structure de données théorique permet de modéliser cette problématique ?

Un graphe

Définir une (ou plusieurs) variable(s)/objet(s) Python pour mémoriser les données de cette carte.

In [1]:
carte = {}

def ajouter_route(dico, depart, arrivee, distance):
    if dico.get(depart) is None:
        dico[depart] = {}
    dico[depart][arrivee] = distance
    
    if dico.get(arrivee) is None:
        dico[arrivee] = {}
    dico[arrivee][depart] = distance

# Ajouter les 14 routes
ajouter_route(carte, 'Lyon', 'Bourgoin-Jallieu', 50)
ajouter_route(carte, 'Lyon', 'Valence', 102)
ajouter_route(carte, 'Valence', 'Voiron', 87)
ajouter_route(carte, 'Valence', 'Villard de Lans', 69)
ajouter_route(carte, 'Villard de Lans', 'Grenoble', 39)
ajouter_route(carte, 'Bourgoin-Jallieu', 'Voiron', 46)
ajouter_route(carte, 'Bourgoin-Jallieu', 'Chambéry', 62)
ajouter_route(carte, 'Voiron', 'Grenoble', 26)
ajouter_route(carte, 'Grenoble', 'Chambéry', 60)
ajouter_route(carte, 'Grenoble', 'Briançon', 115)
ajouter_route(carte, 'Chambéry', 'Albertville', 53)
ajouter_route(carte, 'Chambéry', 'Modane', 103)
ajouter_route(carte, 'Albertville', 'Modane', 90)
ajouter_route(carte, 'Modane', 'Briançon', 60)
carte

{'Lyon': {'Bourgoin-Jallieu': 50, 'Valence': 102},
 'Bourgoin-Jallieu': {'Lyon': 50, 'Voiron': 46, 'Chambéry': 62},
 'Valence': {'Lyon': 102, 'Voiron': 87, 'Villard de Lans': 69},
 'Voiron': {'Valence': 87, 'Bourgoin-Jallieu': 46, 'Grenoble': 26},
 'Villard de Lans': {'Valence': 69, 'Grenoble': 39},
 'Grenoble': {'Villard de Lans': 39,
  'Voiron': 26,
  'Chambéry': 60,
  'Briançon': 115},
 'Chambéry': {'Bourgoin-Jallieu': 62,
  'Grenoble': 60,
  'Albertville': 53,
  'Modane': 103},
 'Briançon': {'Grenoble': 115, 'Modane': 60},
 'Albertville': {'Chambéry': 53, 'Modane': 90},
 'Modane': {'Chambéry': 103, 'Albertville': 90, 'Briançon': 60}}

<div class="alert alert-info">
    
**Piste de réflexion**

Est-ce que votre représentation Python permet ?
- de modifier une distance entre 2 villes voisines.
- d'ajouter (ou enlever) facilement une ville et des routes sur la carte.

Cette version permet une modification rapide d'une distance entre 2 villes voisines ainsi que l'ajout/suppression rapide de routes.

### 2. Algorithme

Quel(s) algorithme(s), déjà rencontré en NSI, peut-être utilisé pour résoudre ce problème ?

Moore-Dijkstra ou Bellman-Ford

Proposer une structure Python pour stocker un itinéraire (étapes et distances) sur cette carte *(prendre l'exemple Lyon-Bourgoin Jallieu-Voiron-Grenoble)*

In [2]:
it = [('Lyon', 0), ('Bourgoin Jallieu', 50), ('Voiron', 96), ('Grenoble', 122)]

Pour utiliser un de ces 2 algorithmes de parcours il faut mémoriser, pour chaque sommet du graphe, les éléments suivants:
- la distance de ce sommet (au sommet de départ pour *Moore-Dijkstra*, au sommet d'arrivée pour *Bellman-Ford*)
- le sommet a-t-il déjà été sélectionné (*visité*)
- le parent du sommet dans l'itinéraire optimal.

In [3]:
# On créé 3 nouveaux dictionnaires pour mémoriser:
# - la distance à la ville de départ
# - les villes visitées
# - la ville précédente dans le parcours
def init_algo(graphe):
    dist = { ville:float('inf') for ville in graphe.keys() }
    vis = { ville:False for ville in graphe.keys() }
    par = { ville:None for ville in graphe.keys() }
    return dist, vis, par
d, v, p = init_algo(carte)
d, v, p

({'Lyon': inf,
  'Bourgoin-Jallieu': inf,
  'Valence': inf,
  'Voiron': inf,
  'Villard de Lans': inf,
  'Grenoble': inf,
  'Chambéry': inf,
  'Briançon': inf,
  'Albertville': inf,
  'Modane': inf},
 {'Lyon': False,
  'Bourgoin-Jallieu': False,
  'Valence': False,
  'Voiron': False,
  'Villard de Lans': False,
  'Grenoble': False,
  'Chambéry': False,
  'Briançon': False,
  'Albertville': False,
  'Modane': False},
 {'Lyon': None,
  'Bourgoin-Jallieu': None,
  'Valence': None,
  'Voiron': None,
  'Villard de Lans': None,
  'Grenoble': None,
  'Chambéry': None,
  'Briançon': None,
  'Albertville': None,
  'Modane': None})

Algorithme de Dijkstra:

In [4]:
def dijkstra(depart, graphe):
    # Récupérer les dictionnaires utiles pour l'algo
    distance, visite, parent = init_algo(graphe)
    
    # Ville de départ
    assert graphe.get(depart) is not None, f"La ville de départ {depart} n'existe pas."
    distance[depart] = 0 
    
    while sum(visite.values())!=len(graphe.keys()): # Tant qu'il reste des sommets non visités
        
        # 1) Sélectionner la (première) ville non visitée de distance minimale
        dist_min = float('inf')
        ville_min = None
        for ville in graphe.keys():
            if not visite[ville] and distance[ville] < dist_min:
                ville_min = ville
                dist_min = distance[ville]
        visite[ville_min] = True

        # 2) Ajouter les voisins immédiats de distance minimale 
        # en tant qu'enfant
        for ville_dst in graphe[ville_min].keys():
            if distance[ville_min] + graphe[ville_min][ville_dst] < distance[ville_dst]:
                distance[ville_dst] = distance[ville_min] + graphe[ville_min][ville_dst]
                parent[ville_dst] = ville_min
    return distance, parent

Écrire une fonction qui retourne l'itinéraire optimal entre 2 villes données:

In [5]:
def itineraire(depart, arrivee, graphe):
    # Lancer l'algorithme (non nécessaire si déjà calculé pour ce départ)
    distance, parent = dijkstra(depart,graphe)
    
    # Ville d'arrivée
    assert graphe.get(arrivee) is not None, f"La ville d'arrivee {arrivee} n'existe pas."
    
    # Construire le parcours
    parcours = []
    ville = arrivee
    while ville!=depart:
        parcours.insert(0,(ville, distance[ville]))
        ville = parent[ville]
    parcours.insert(0,(depart, 0))
    return parcours

In [6]:
itineraire('Lyon', 'Grenoble', carte)

[('Lyon', 0), ('Bourgoin-Jallieu', 50), ('Voiron', 96), ('Grenoble', 122)]

Écrire le programme de calcul d'itinéraire version console

## II. Version graphique

Cahier des charges:
- L'IHM est écrite en langage HTML (+JavaScript, CSS)
- L'exécution de l'algorithme s'effectue sur un serveur Flask en langage Python.

## III. Pistes d'amélioration

- Augmenter le nombre de villes sur la carte.
- Utiliser d'autres pondérations (temps de parcours&hellip;)
- &hellip;